# Mining Text to Find Meaningful Data

Text offers plenty of possibilities for analysis. We can extract meaning from *unstructured data* -- paragraphs of text in speeches, reports, press releases, & other documents -- by transforming it into *structured data*, in rows & columns in a table. We can use advanced text analysis features with PostgreSQL's full-text search. With these techniques, ordinary text can reveal facts or trends that might otherwise remain hidden.

---

# Formatting Text Using String Functions

PostgreSQL has more than 50 built-in string functions that handle routine but necessary tasks, such as capitalising letters, combining strings, & removing unwanted spaces. Some are part of ANSI SQL standard, & others are specific to PostgreSQL. You can find the complete list of string functions at [https://www.postgresql.org/docs/current/functions-string.html](https://www.postgresql.org/docs/current/functions-string.html).

## Case Formatting

The capitalisation functions format the text's case. The `upper(string)` function capitalises all alphabetical characters of a string passed to it. Nonalphabetic characters, such as numbers, remain unchanged. For example, `upper('Neal7')` returns `NEAL7`. The `lower(string)` function lowercases all alphabetical characters while keeping nonalphabetic characters unchanged. For example, `lower('Randy')` returns `randy`.

The `initcap(string)` function capitalises the first letter of each word. For example, `initcap('a the end of the day')` returns `At The End Of The Day`. This function can be handy for formatting titles of books or movies, but because it doesn't recognise acronyms, it's not always the perfect solution. For example, `initcap('Practical SQL')` returns `Practical Sql`, because it doesn't recognise SQL as an acronym.

The `upper()` & `lower()` functions are ANSI SQL standard commands, but `initcap()` is PostgreSQL-specific.

## Character Information

Several functions return data about the string & are helpful on their own or combined with other functions. The `char_length(string)` function returns the number of characters in a string, including any spaces. For example, `char_length(' Pat ')` returns a value of `5`, because the three letters in `Pat` & the spaces on either end total five characters. You can also use non-ANSI SQL function `length(string)` to count strings, which has a variant that lets you count the length of binary strings.

```
SELECT char_length(' Pat ');
```

<img src = "char_length().png" width = "600" style = "margin:auto"/>

The `position(substring in string)` function returns the location of the substring characters in the string. For example, `position(', ' in 'Tan, Bella')` returns `4`, because the comma & space characters (`, `) specified in the substring passed as the first parameter starting at the fourth index position in the main string `Tan, Bella`. 

Both `char_length()` & `position()` are in the ANSI SQL standard.

## Removing Characters

The `trim(characters from string)` function removes characters from the beginning & end of a string. To declare one or more characters to remove, add them to the function followed by the keyword `from` & the string you want to change. Options to remove `leading` characters (at the front of the string), `trailing` characters (at the end of the string), or `both` make this function super flexible.

For example, `trim('s' from 'socks')` removes `s` characters at the beginning & end, returning `ock`. To remove only the `s` at the end of the string, add the `trailing` keyword before the character to trim: `trim(trailing 's' from 'socks')` returns `sock`.

If you don't specify any characters to remove, `trim()` removes spaces at either end of the string by default. For example, `trim(' Pat ')` returns `Pat` without the leading or trailing spaces. To confirm the length of the trimmed string, we can nest `trim()` inside `char_length()` like this:

```
SELECT char_length(trim(' Pat '));
```

<img src = "trim().png" width = "600" style = "margin:auto"/>

This query should return `3`, the number of letters in `Pat`, which is the result of `trim(' Pat ')`. The `ltrim(string, characters)` & `rtrim(string, characters)` functions are PostgreSQL-specific variation of the `trim()` function. They remove characters from the left or right ends of a string. For example, `rtrim('socks', 's')` returns `sock` by removing only the `s` on the right end of the string.

## Extracting & Replacing Characters

The `left(string, number)` & `right(string, number)` functions, both ANSI SQL standard, extract & return selected characters from a string. For example, to get just the `703` area code from the phone number `703-555-1212`, use `left('703-555-1212', 3)` to specify `right('703-555-1212', 8)` returns eight characters from the right: `555-1212`.

To substitute characters in a string, use the `replace(string, from, to)` function. To change `bat` to `cat`, for example, you would use `replace('bat', 'b', 'c')` to specify that you want to replace the `b` in `bat` with a `c`.

---

# Matching Text Patterns with Regular Expressions

*Regular expressions* (or *regex*) are a type of notational language that describes text patterns. If you have a string with a noticeable pattern (say, four digits followed by a hyphen & then two more digits), you can write a regular expression that matches the pattern. You can then use the notation in a `WHERE` clause to filter rows by the pattern or use regular expression functions to extract & wrangle text that contains the same pattern.

## Regular Expression Notation

Matching letters & numbers using regular expression notation is straightforward because letters & numbers (& certain symbols) are literals that indicate the same characters. For example `Al` matches the first two characters in `Alicia`.

For more complex patterns, you can use combinations of the regular expression elements in the table below:

|Expression|Description|
|:---|:---|
|.|A dot is a wildcard that finds any character except a newline.|
|[Fgz]|Any character in the square brackets. Here, *F*, *g*, or *z*.|
|[a-z]|A range of characters. Here, lowercase *a* to *z*.|
|[^a-z]|The caret negates the match. Here, not lowercase *a* to *z*.|
|\w|Any word character or underscore. Same as `[A-Za-z0-9_]`.|
|\d|Any digit.|
|\s|A space.|
|\t|Tab character.|
|\n|Newline character.|
|\r|Carriage return character.|
|^|Match at the start of a string.|
|$|Match at the end of a string.|
|?|Get the preceding match zero or one time.|
|*|Get the preceding match zero or more times.|
|+|Get the preceding match one or more times.|
|(m)|Get the preceding match exactly `m` times.|
|(m, n)|Get the preceding match between `m` & `n` times.|
|a &#124; b|The pipe denotes alternation. Find either `a` or `b`.|
|( )|Create & report a capture group or set precedence.|
|(?: )|Negate the reporting of a capture group.|

Using these regular expressions, you can match various characters & indicate how many times & where to match them. For example, placing characters inside square brackets (`[]`) lets you match any single character or a range. So, `[FGz]` matches a single `F`, `G`, or `z`, whereas `[A-Za-z]` will match any uppercase or lowercase letter.

The backlash (`\`) precedes a designator for special characters, such as a tab (`\t`), digit (`\d`), or newline (`\n`), which is a line ending character in text files.

There are several ways to indicate how many times to match a character. Placing a number inside curly brackets indicate you want to match it that many times. For example `\d{4}` matches four digits in a row, & `\d{1, 4}` matches one to four digits.

The `?`, `*`, & `+` characters provide a useful shorthand notation for the number of matches you want. The plus sign (`+`) after a character indicates to match it one or more times, for example. So, the expression `a+` would find the `aa` characters in the string `aardvark`.

Additionally, parentheses indicate a *capture group*, which you can use to identify a portion of the entire matched expression. For example, if you were hunting for an `HH:MM:SS` time format in text & wanted to report only the hour, you could use an expression such as `(\d{2}):\d{2}:\d{2}`. This looks like two digits (`\d{2}`) of the hour followed by a colon, another two digits for the minutes & a colon, & then the two-digit seconds. By placing the first `\d{2}` inside parentheses, you can extract only those two digits, even though the entire expression matches the full time.

The below shows examples of combining regular expressions to capture different portions of the sentence "The game starts at 7 p.m. on May 2, 2024".

|Expression|What it matches|Result|
|:---|:---|:---|
|.+|Any character one or more times|`The game starts at 7 p.m. on May 2, 2024.`|
|\d{1, 2} (?:a.m. &#124; p.m.)|On or two digits followed by a space & *a.m.* or *p.m.* in a noncapture group|`7 p.m.`|
|^\w+|One or more word characters at the start|`The`|
|\w+.$|One or more word characters followed by any character at the end|`2024.`|
|May &#124; June|Either of the words *May* or *June*|`May`|
|\d{4}|Four digits|`2024`|
|May \d \d{4}|*May* followed by a space, digit, comma, space, & four digits|`May 2, 2024`|

These results show the usefullness of regular expressions for matching the parts of the string that interest us. For example, to find the time, we use the expression `\d{1, 2} (?:a.m.|p.m.)` to look for either one or two digits because the time could be a single or double digit followed by a space. Then we look for either `a.m.` or `p.m.`; the pipe symbol separating the terms indicates the either-or condition, & placing them in parentheses separates the logic from the rest of the expression. We need the `?:` symbol to indicate that we don't want to treat the terms inside the parentheses as a capture group, which would report `a.m.` or `p.m.` only. The `?:` ensures that the full match will be returned.

You can use any of these regular expressions by placing the text & regular expression inside the `substring(string from pattern)` function to return the matched text. For example, to find the four-digit year, use the following query:

```
SELECT substring('The game starts at 7 p.m. on May 2, 2024' from '\d{4}');
```

This query should return `2024`, because we specified the pattern should look for four digits in a row, & 2024 is the only portion of this string that matches these criteria.

<img src = "substring() & CTEs.png" width = "600" style = "margin:auto"/>

## Using Regular Expressions with WHERE

We've filtered queries using `LIKE` & `ILIKE` in `WHERE` clauses. In this section, we'll learn to use regular expressions in `WHERE` clauses so we can perform more complex matches.

We use a tilde (`~`) to make a case-sensitive match on a regular expression & a tilde-asterisk (`~*`) to perform a case-insensitive match. You can negate either expression by adding an exclamation point in front. For example, `!~*` indicates *not* to match a regular expression that is case-insensitive. The query below shows how this works using the 2019 US Census estimates `us_counties_pop_est_2019` table from previous exercises.

```
SELECT county_name
FROM us_counties_pop_est_2019
WHERE county_name ~* '(lade|lare)'
ORDER BY county_name;

SELECT county_name
FROM us_counties_pop_est_2019
WHERE county_name ~* 'ash' AND county_name !~ 'Wash'
ORDER BY county_name;
```

The first `WHERE` clause uses the tilde-asterisk (`~*`) to perform a case-insensitive match on the regular expression `(lade|lare)` to find any county names that contain either the letters `lade` or `lare`. The results show eight rows:

<img src = "Using Regular Expressions in a WHERE Clause 1.png" width = "600" style = "margin:auto"/>

As you can see, each county name includes the letters `lade` or `lare`.

The second `WHERE` clause uses the tilde-asterisk (`~*`) as well as a negated tilde (`!~`) to find county names containing the letters `ash` but excluding those that include `Wash`. This query should return the following:

<img src = "Using Regular Expressions in a WHERE Clause 2.png" width = "600" style = "margin:auto"/>

All nine counties in this output have names that contain the letters `ash`, but none have `Wash`.

These are fairly simple examples, but you can do more complex matches using regular expressions you wouldn't be able to perform with the wildcards available with just `LIKE` & `ILIKE`.

## Regular Expression Functions to Replace or Split Text

The query below shows three regular expression functions that replace & split text.

```
SELECT regexp_replace('05/12/2024', '\d{4}', '2023');

SELECT regexp_split_to_table(
           'Four, score, and, seven, years, ago', ', ');

SELECT regexp_split_to_array('Phil Mike Tony Steve', ' ');
```

The `regexp_replace(string, pattern, replacement text)` function lets you substitute a matched pattern with replacement text. In the example, we're searching the date string `05/12/2024` for any set of four digits in a row using `\d{4}`. When found, we replace them with the replacement text `2023`. The result of that query is `05/12/2023` returned as text.

<img src = "Regular Expression Function to Replace Text.png" width = "600" style = "margin:auto"/>

The `regexp_split_to_table(string, pattern)` function splits delimited text into rows. We use this function to split the string `'Four, score, and, seven, years, ago'` on commas, resulting in a set of rows that has one word in each row:

<img src = "Regular Expression Function to Split Text 1.png" width = "600" style = "margin:auto"/>

The `regexp_split_to_array(string, pattern)` function splits delimited text into an array. The example splits the string `Phil Mike Tony Steve` on spaces, returning a text array that should look like this in pgAdmin:

<img src = "Regular Expression Function to Split Text 2.png" width = "600" style = "margin:auto"/>

The `text[]` notation in pgAdmin's column header along with curly brackets around the results confirms that this is indeed an array type, which provides another means of analysis. For example, we can use a function such as `array_length()` to count the number of words.

```
SELECT array_length(regexp_split_to_array(
           'Phil Mike Tony Steve', ' '), 1);
```

The array that `regexp_split_to_array()` produces is one-dimensional; that is, the result contains one list of names. Arrays can have additional dimensions -- for example, a two-dimensional array can represent a matrix with rows & columns. Thus, here we pass `1` as a second argument to `array_length()` indicating we want the length of the first (& only) dimension of the array. The query should return `4` because the array has four elements.

<img src = "Finding an Array Length.png" width = "600" style = "margin:auto"/>

If you can identify a pattern in the text, you can use a combination of regular expression symbols to locate it. This technique is particularly useful when you have repeating patterns in text that you want to turn into a set of data to analyse.

## Turning Text to Data with Regular Expression Functions

A sheriff's department in one of the Washington, DC suburbs publishes daily reports that detail the date, time, location, & description of incidents the department investigates. These reports would be great to analyse, except they post the information in Microsoft Word documents saved as PDF files, which is not the friendliest format for importing into a database.

If I copy & paste incidents from the PDF into a text editor, the result is blocks of text that look something like so:

```
4/16/17-4/17/17
2100-0900 hrs.
46000 Block Ashmere Sq. 4 Sterling
Larceny: The victim reported that a bicycle was stolen from their opened garage door during the overnight hours.
C0170006614

  04/10/17     
  1605 hrs.
  21800 block Newlin Mill Rd.
  Middleburg
  Larceny: A license plate was reported
  stolen from a vehicle.
  SO170006250
```

Each block of text includes dates, times, a street address, city or town, the type of crime, & a description of the incident. The last piece of information is a code that might be a unique ID for the incident, although we'd have to check with the sheriff's department to be sure. There are slight inconsistencies. For example, the first block of text has two dates (`4/16/17 - 4/17/17`) & two times (`2100 - 0900 hrs.`), meaning the exact time of the incident is unknown & likely occurred within that time span. The second block has one date & time.

If you compile these reports regularly, you can expect to find some good insights that could answer important questions: Where do crimes tend to occur? Which crime types occur most frequently? Do they happen more often on weekends or weekdays? We'll need to extract the text into table columns using regular expressions.

### Creating a Table for Crime Reports

We have five crime incidents in a file named *crime_reports.csv*. Build a table tht has a column for each data element you can parse from the text using a regular expression.

```
CREATE TABLE crime_reports (
    crime_id integer PRIMARY KEY
        GENERATED ALWAYS AS IDENTITY,
    case_number text,
    date_1 timestamptz,
    date_2 timestamptz,
    street text,
    city text,
    crime_type text,
    description text,
    original_text text NOT NULL
);

COPY crime_reports (original_text)
FROM '/YourDirectory/crime_reports.csv'
WITH (FORMAT CSV, HEADER OFF, QUOTE '"');
```

Run the `CREATE TABLE` statment & then use `COPY` to load the text into the column `original_text`. The rest of the columsn will be `NULL` until we fill them.

When you run `SELECT original_text FROM crime_reports;` in pgAdmin, the results should display five rows & the first several words of each report. When you double click any cell, pgAdmin shows all the text in that row.

<img src = "Displaying Additional Text in the pgAdmin Results Grid.png" width = "600" style = "margin:auto"/>

### Matching Crime Report Date Patterns

The first piece of data we want to extract from `original_text` is the date or dates of the crime. Most reports have one date, although one has two. The reports also have associated times, & we'll combine the extracted date & time into a timestamp. We'll fill `date_1` with each report's first (or only) date & time. If a second date or second time exists, we'll add it to `date_2`.

We'll use the `regexp_match(string, pattern)` function, which is similar to `substring()` with a few exceptions. One is that it returns each match as text in an array. Also, if there are no matches, it returns `NULL`. As you might recall, you can use an array to past a list of values into `percentile_cont()` function to calculate quartiles.

Let's use `regexp_match()` to find dates in each of the five incidents. The general pattern to match is `MM/DD/YY`, although there may be one or two digits for both the month & date. Here's a regular expression that matches the pattern:

```
\d{1,2}\/d{1,2}\/\d{2}
```

In this expresison, the first `\d{1, 2}` indicates the month. The numbers inside the curly brackets specify that you want at least one digit & at most two digits. Next, you want to look for a forward slash(`/`), but because a forward slash can have a special meaning in regular expressions, you must *escape* that character by placing a backslash (`\`) in front of it, like this `\/`. Escaping a character in this context simply means we want to treat it as a literal rather than letting it take on special meaning. So, the combination of the backslash & forward slash (`\/`) indicates you want a forward slash.

Another `\d{1,2}` follows for a single- or double-digit day of the month. The expression ends with a second escaped forward slash & `\d{2}` to indicate the two-digit year. Let's pass this expression `\d{1,2}\/\d{1,2}\/\d{2}` to `regexp_match()`:

```
SELECT crime_id,
       regexp_match(original_text,
           '\d{1, 2}\/\d{1, 2}\/\d{2}')
FROM crime_reports
ORDER BY crime_id;
```

The results should look like this:

<img src = "Using regexp_match() to Find the First Date.png" width = "600" style = "margin:auto"/>

Note that each row shows the first date listed for the incident, because `regexp_match()` returns the first match it finds by default. Also note that each date is enclosed in curly brackets. That's PostgreSQL indicating that `regexp_match()` returns each result as an array type, or list of elements.

### Matching the Second Date When Present

We've successfully matched the first date from each report. But recall that one of the five incidents has a second date. To find & display all the dates in the text, you must use the related `regexp_matches()` function & pass in an option in the form of flag `g`.

```
SELECT crime_id,
       regexp_matches(original_text,
           '\d{1,2}\/\d{1,2}\/\d{2}', 'g')
FROM crime_reports
ORDER BY crime_id;
```

Any time a crime report has a second date, we want to load it & the associated time into the `date_2` column. Although adding the `g` flag shows us all the dates, to extract just the second date in the report, we can use the pattern we always see when two dates exist. From our initial view of the data, we could see that the first block of text showed two dates separated by a hyphen:

<img src = "Displaying Additional Text in the pgAdmin Results Grid.png" width = "600" style = "margin:auto"/>

This means we can switch back to `regexp_match()` & write a regular expression to look for a hyphen followed by a date.

```
SELECT crime_id,
       regexp_match(original_text, '-\d{1,2}\/\d{1,2}\/\d{2}')
FROM crime_reports
ORDER BY crime_id;
```

Although this query finds the second date in the first item (& returns a `NULL` for the rest), there's an unintended consequence: it displays the hyphen along with it.

<img src = "Using regexp_match() to Find the Second Date.png" width = "600" style = "margin:auto"/>

You don't want to include the hyphen, because it's an invalid format for the `timestamp` data type. Fortunately, you can specify the exact part of the regular expression you want to return by placing parentheses aroudn it to create a capture group, like this:

```
-(\d{1,2}\/\d{1,2}\/\d{2})
```

This notation returns only the part of the regular expression you want.

```
SELECT crime_id,
       regexp_match(original_text,
           '-(\d{1,2}\/\d{1,2}\/\d{2})')
FROM crime_reports
ORDER BY crime_id;
```

The query should return just the second date without the leading hyphen as shown:

<img src = "Using a Capture Group to Return Only the Date.png" width = "600" style = "margin:auto"/>

### Matching Additional Crime Report Elements

Here are the expressions for capturing times, addresses, crime types, descriptions, & case numbers from the crime reports.

### First Hour \/\d{2}\n(\d{4})

The first hour, which is the hour the crime was committed or the start of the time range, always follows the date in each crime report, like so:

```
4/16/17-4/17/17
2100-0900 hrs.
```

To find the first hour, we start with an escaped forward slash & `\d{2}`, which represents the two-digit year preceding the second date (`17`). The `\n` character indicates the newline because the hour always starts on a new line, & `\d{4}` represents the four-digit hour (`2100`). Because you just want to return the four digits, we put `\d{4}` inside parentheses as a capture group.

```
SELECT crime_id,
       regexp_match(original_text,
           '\/\d{2}\n(\d{4})')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "First Hour of Crime Reports.png" width = "600" style = "margin:auto"/>

### Second Hour \/\d{2}\n\d{4}-(\d{4})

If the second hour exists, it will follow a hyphen, so we add a hyphen & another `\d{4}` to the expression we just created for the first hour. Again, the second `\d{4}` goes inside a cpature group, because `0900` is the only hour we want to return.

```
SELECT crime_id,
       regexp_match(original_text,
           '\/\d{2}\n\d{4}-(\d{4})')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "Second Hour of Crime Reports.png" width = "600" style = "margin:auto"/>

### Street hrs.\n(\d+ .+(?:Sq.|Plz.|Dr.|Ter.|Rd.))

In the crime reports data, the street always follows the time's `hrs.` designation & a newline (`\n`), like so:

```
04/10/17
1605 hrs.
21800 block Newlin Mill Rd.
```

The street address always starts with some number that varies in length & ends with an abbreviated suffix of some kind. To describe this pattern, we use `\d+` to match any digit that appears one or more times. Then we specify a space & look for any character one or more times using the dot wildcard & plus sign (`.+`) notation. The expression ends with a series of terms separated by the alternation pipe symbol that looks like this: `(?:Sq.|Plz.|Dr.|Ter.|Rd.)`. The terms are inside parentheses, so the expression will match one or another of those terms. When we group terms like this, if we don't want the parentheses to act as a capture group, we need to add `?:` to negate that effect.

```
SELECT crime_id,
       regexp_match(original_text,
           'hrs.\n(\d+ .+(?:Sq.|Plz.|Dr.|Ter.|Rd.))')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "Street from Crime Reports.png" width = "600" style = "margin:auto"/>

### City (?:Sq.|Plz.|Dr.|Ter.|Rd.)\n(\w+ \w+|\w+)\n

Because the city always follows the street suffix, we reuse the terms separated by the alternation symbol we just created for the street. We follow that with a new line (`\n`) & then use a capture group to look for two words or one word (`\w+ \w+|\w+`) before a final newline, because a town or city name can be more than one single word.

```
SELECT crime_id,
       regexp_match(original_text,
           '(?:Sq.|Plz.|Dr.|Ter.|Rd.)\n(\w+ \w+|\w+)\n')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "City of Crime Reports.png" width = "600" style = "margin:auto"/>

### Crime Type \n(?:\w+ \w+|\w+)\n(.*):

The type of crime always precedes a colon (the only time a colon is used in each report) & might consist of one or more words, like this:

```
Middleburg
Larceny: A license plate was reported
stolen from a vehicle.
SO170006250
```

To create an expression that matches this pattern, we follow a newline with a nonreporting capture group that looks for the one-or two-word city. Then we add another newline & match any character that occurs zero or more times before a colon using `(.*):`.

```
SELECT crime_id,
       regexp_match(original_text,
           '\n(?:\w+ \w+|\w+)\n(.*):')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "Crime Type of Crime Reports.png" width = "600" style = "margin:auto"/>

### Description :\s(.+) (?:C0|SO)

The crime description always comes between the colon after the crime type & the case number. The expression starts with the colon, a space character (`\s`), & then a capture group to find any character that appears one or more times using the `.+` notation. The nonreporting capture group `(?:C0|SO)` tells the program to step looking when it encounters either `C0` or `SO`, the tw character pairs that start each case number. We have to do this because the description might have one or more line breaks.

```
SELECT crime_id,
       regexp_match(original_text,
           ':\s(.+)(?:C0|SO)')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "Description of Crime Reports.png" width = "600" style = "margin:auto"/>

### Case Number (?:C0|SO)[0-9]+

The case number starts with either `C0` or `SO`, followed by a set of digits. To match this pattern, the expression looks for either `C0` or `SO` in a nonreporting capture group followed by any digit from 0 to 9 that occurs one or more times using the `[0-9]` range notation.

```
SELECT crime_id,
       regexp_match(original_text,
           '(?:C0|SO)[0-9]+')
FROM crime_reports
ORDER BY crime_id;
```

<img src = "Case Number of Crime Reports.png" width = "600" style = "margin:auto"/>

Now let's pass some of these regular expressions to `regexp_match()` to see them in action. The query below shows a sample `regexp_match()` query that retrieves the case number, first date, crime type, & city.

```
SELECT regexp_match(original_text,
           '(?:C0|SO)[0-9]+') AS case_number,
       regexp_match(original_text,
           '\d{1,2}\/\d{1,2}\/\d{2}') AS date_1,
       regexp_match(original_text,
           '\n(?:\w+ \w+|\w+)\n(.*):') AS crime_type,
       regexp_match(original_text,
           '(?:Sq.|Plz.|Dr.|Ter.|Rd.)\n(\w+ \w+|\w+)\n')
           AS city
FROM crime_reports
ORDER BY crime_id;
```

The result should look like this:

<img src = "Matching Case Number, Date, Crime Type, & City.png" width = "600" style = "margin:auto"/>

After all that wrangling, we've transformed the text into a structure that is more suitable for analysis. Of course, you would have to include many more incidents to count the frequency of crime type by city or by the number of crimes per month to identify any trends.

To load each parse element into the table's columns, we'll create an `UPDATE` query. But before we can insert the text into a column, we'll need to learn how to extract the text from the array that `regexp_match()` returns.

### Extracting Text from the regexp_match() Result

`regexp_match()` returns data in an array type containing text. Two clues reveal that these are array types. The first is that the data type designation in the column header shows `text[]` instead of `text`. The second is that each result is surrounded by curly brackets. Let's revisit our pgAdmin displays the results of the previous query:

<img src = "Matching Case Number, Date, Crime Type, & City.png" width = "600" style = "margin:auto"/>

The `crime_reports` columns we want to update are not array types, so rather than passing in the array values returned by `regexp_match()`, we need to extract the values from the array first. We do this by using array notation, as shown below:

```
SELECT crime_id,
       (regexp_match(original_text,
           '(?:C0|SO)[0-9]+'))[1] AS case_number
FROM crime_reports
ORDER BY crime_id;
```

First, we wrap the `regexp_match()` function in parentheses. Then, at the end, we provide a value of `1`, which represents the first element in the array, enclosed in square brackets. The query should produce the following results:

<img src = "Retrieving a Value From Within An Array.png" width = "600" style = "margin:auto"/>

Now the data type designation in the pgAdmin column header should show `text` instead of `text[]`, & the values are no longer enclosed in curly brackets. We can now insert these values into `crime_reports` using an `UPDATE` query.

### Updating the crime_reports Table with Extracted Data

To start updating columns in `crime_reports`, the query below combines the extracted first date & time into a single `timestamp` value for the column `date_1`.

```
UPDATE crime_reports
SET date_1 = (
    (regexp_match(original_text,
    '\d{1,2}\/\d{1,2}\/\d{2}'))[1]
        || ' ' ||
    (regexp_match(original_text,
    '\/\d{2}\n(\d{4})'))[1]
        ||' US/Eastern'
)::timestamptz
RETURNING crime_id, date_1, original_text;
```

Because the `date_1` column is of type `timestamp`, we must provide an input in that data type. To do that, we'll use the PostgreSQL double-pipe (`||`) concatenation operator to combine the extracted date & time in a format that's acceptable for `timestamp with time zone` input. In the `SET` clause, we start with the regex pattern that matches the first date. Next, we concatenate the date with a space using two single quotes & repeat the concatenation operator. This step combines the date with a space before connecting it to the regex pattern that matches the time. Then we include the time zone for Washington, DC, area by concatenating that at the end of the string using the `US/Eastern` designation. Concatenating these elements creates a string in the pattern of `MM/DD/YY HH:MM TIMEZONE`, which is acceptable as a `timestamp` input. We cast the string to a `timestamp with time zone` data type using the PostgreSQL double-colon shorthand & the `timestamptz` abbreviation.

When you run the `UPDATE`, the `RETURNING` clause will display the columns we specify from the updated rows, including the now-filled `date_1` column alongside a portion of the `original_text` column, like this:

<img src = "Updating the crime_reports date_1 Column.png" width = "600" style = "margin:auto"/>

At a glance, you can see that `date_1` accurately captures the first date & time that appears in the original text & puts it into a format that we can analyse -- quantifying, for example, which times of day crimes most often occur. Note that if you're not in the Eastern time zone, the timestamps will instead reflect your pgAdmin client's time zone. Also, in pgAdmin, you may need to double-click a cell in the `original_text` column to see the full text.

### Using CASE to Handle Special Instances

We could write an `UPDATE` statement for each remaining data element, but combining those statements into one would be more efficient. The query below updates all the `crime_reports` columns using a single statement while handling inconsistent values in the data.

```
UPDATE crime_reports
SET date_1 = (
        (regexp_match(original_text,
        '\d{1,2}\/\d{1,2}\/\d{2}'))[1]
            || ' ' ||
        (regexp_match(original_text,
        '\/\d{2}\n(\d{4})'))[1]
            ||' US/Eastern'
              )::timestamptz,
    date_2 =
        CASE
            WHEN (SELECT regexp_match(original_text,
                 '-(\d{1,2}\/\d{1,2}\/\d{2})') IS NULL)
                 AND (SELECT regexp_match(original_text,
                     '\/\d{2}\n\d{4}-(\d{4})') IS NOT NULL)
            THEN ((regexp_match(original_text,
                  '\d{1,2}\/\d{1,2}\/\d{2}'))[1]
                      || ' ' ||
                  (regexp_match(original_text,
                  '\/\d{2}\n\d{4}-(\d{4})'))[1]
                      ||' US/Eastern'
                  )::timestamptz
            WHEN (SELECT regexp_match(original_text,
                 '-(\d{1,2}\/\d{1,2}\/\d{2})') IS NOT NULL)
                 AND (SELECT regexp_match(original_text,
                     '\/\d{2}\n\d{4}-(\d{4})') IS NOT NULL)
            THEN ((regexp_match(original_text,
                  '-(\d{1,2}\/\d{1,2}\/\d{1,2})'))[1]
                      || ' ' ||
                  (regexp_match(original_text,
                  '\/\d{2}\n\d{4}-(\d{4})'))[1]
                      ||' US/Eastern'
                  )::timestamptz
        END,
    street = (regexp_match(original_text,
        'hrs.\n(\d+ .+(?:Sq.|Plz.|Dr.|Ter.|Rd.))'))[1],
    city = (regexp_match(original_text,
        '(?:Sq.|Plz.|Dr.|Ter.|Rd.)\n(\w+ \w+|\w+)\n'))[1],
    crime_type = (regexp_match(original_text,
        '\n(?:\w+ \w+|\w+)\n(.*):'))[1],
    description = (regexp_match(original_text,
        ':\s(.+)(?:C0|SO)'))[1],
    case_number = (regexp_match(original_text,
        '(?:C0|SO)[0-9]+'))[1];
```

This `UPDATE` statement might look intimidating, but it's not if we break it down by column. First, we update the `date_1` column like in the previous query. But to update `date_2`, we need to account for the inconsistent presence of a second date & time. In our limited dataset, there are three probabilities:

A second hour exists but not a second date. This occurs when a report covers a range of hours on one date.

A second date & second hour exist. This occurs when a report covers more than one date. 

Neither a second date nor a second hour exists.

To insert the correct value in `date_2` for each scenario, we use a `CASE` statement to test for each possibility. After the `CASE` keyword, we use a series of `WHEN ... THEN` statements to check for the first two conditions & provide the value to insert; if neither condition exists, the `CASE` statement will by default return a `NULL`.

The first `WHEN` statement checks whether `regexp_match()` returns a `NULL` for the second date & a value for the second hour (using `IS NOT NULL`). If that condition evaluates as `true`, the `THEN` statement concatenates the first date with the second hour to create a timestamp for the update.

The second `WHEN` statement checks that `regexp_match()` returns a value for the second hour & second date. If `true`, the `THEN` statement concatenates the second date with the second hour to create a timestamp.

If neither of the two `WHEN` statements return `true`, the `CASE` statement will return a `NULL` because there is only a first date & first time.

When we run the full query, PostgreSQL should report `UPDATE 5`. Now that you've updated all the columns with the appropriate data while accounting for elements that have additional data, we can examine all the columns of the table & find the parse elements from `original_text`.

```
SELECT date_1, street, city, crime_type
FROM crime_reports
ORDER BY crime_id;
```

The results the query should show a nicely organised set of data that looks like this:

<img src = "Viewing Selected Crime Data.png" width = "600" style = "margin:auto"/>

### The Value of the Process

Writing regular expressions & coding a query to update a table can take time, but there is value to identifying & collecting data this way. In fact, some of the best datasets are ones you build yourself. Everyone can download the same datasets, but the ones you build are yours alone.

Also, after you set up your database & queries, you can use them again & again. In this example, you can collect crime reports every date (either by hand or by automating downloads using a programming language such as Python) for an ongoing dataset that you can mine continually for trends.

---

# Full-Text Search in PostgreSQL

PostgreSQL's comes with a powerful full-text search engine that adds capabilities for searching large amounts of text, similar to online search tools & technology that powers search on research databases, such as Factiva. 

Let's start with the data types unique to full-text search.

## Text Search Data Types

PostgreSQL's implementation of text search includes two data types. The `tsvector` data type represents the text to be searched & to be stored in a normalised form. The `tsquery` data type represents the search query terms & operators. 

### Storing Text as Lexemes with tsvector

The `tsvector` data type reduces text to a sorted list of *lexemes*, which are linguistic units in a given language. It's helpful to think of lexemes as word roots without the variations created by suffixes. For example, a `tsvector` type column would store the words *washes*, *washed*, & *washing* as the lexeme *wash* while noting each word's position in the original text. Converting text to `tsvector` also removes small *stop words* that usually don't play a role in search; such as *the* or *it*.

To see how this data type works, let's convert a string to `tsvector` format. The query below uses the PostgreSQL search function `to_tsvector()`, which normalises the text "I am walking across the sitting room to sit with you" to lexemes using the `english` language search configuration.

```
SELECT to_tsvector('english',
           'I am walking across the sitting
           room to sit with you.');
```

Execute the query & it should return the following output in the `tsvector` data type:

<img src = "Convert Text to tsvector Data.png" width = "600" style = "margin:auto"/>

The `to_tsvector()` function reduces the number of words from eleven to four, eliminating words such as *I*, *am*, & *the*, which are not helpful search terms. The function removes suffixes, changing *walking* to *walk* & *sitting* to *sit*. It orders the words alphabetically, & the number following each colon indicates its position in the original string, taking stop words into account. Not that *sit* is recognised as being in two positions, one for *sitting* & one for *sit*.

### Creating the Search Terms with tsquery

The `tsquery` data type represents the full-text search query, again optimised as lexemes. It also provides operators for controlling the search. Examples of operators include the ampersand (`&`) for AND, the pipe symbol (`|`) for OR, & the exclamation point (`!`) for NOT. The `<->` followed by operator lets you search for adjacent words or words a certain distance apart.

The query below shows how the `to_tsquery()` function converts search terms to the `tsquery` data type.

```
SELECT to_tsquery('english',
           'walking & sitting');
```

After running the code, you should see that the resulting `tsquery` data type has normalised the terms into lexemes, which match the format of the data to search:

<img src = "Converting Search Terms to tsquery Data.png" width = "600" style = "margin:auto"/>

Now you can use terms stored as `tsquery` to search text optimised as `tsvector`.

### Using the @@ Match Operator for Searching

With the text & search terms converted to the full-text search data types, you can use the double at sign (`@@`) match operator to check whether a query matches text. The first query in the query below uses `to_tsquery()` to evaluate whether the text contains both *walking* & *sitting*, which we combine with the `&` operator. It returns a Boolean value of `true` because the lexemes of both *walking* & *sitting* are present in the text converted to `to_tsvector()`.

```
SELECT to_tsvector('english',
           'I am walking across the sitting
           room') @@ to_tsquery('english',
           'walking & sitting');

SELECT to_tsvector('english',
           'I am walking across the sitting
           room') @@ to_tsquery('english',
           'walking & running');
```

<img src = "Querying a tsvector Type with a tsquery 1.png" width = "600" style = "margin:auto"/>

However, the second query returns `false` because both *walking* & *running* are not present in the text. Now let's build a table for searching the speeches.

<img src = "Querying a tsvector Type with a tsquery 2.png" width = "600" style = "margin:auto"/>

## Creating a Table for Full-Text Search

The query below creates & fills `president_speeches` with a column for the original text as well as a column of type `tsvector`. After the import, we'll convert the speech text to the `tsvector` data type. Note the accommodations for setting up the CSV file, the `WITH` clause in the `COPY` statement includes a pipe-delimiter & uses an ampersand for quoting.

```
CREATE TABLE president_speeches (
    president text NOT NULL,
    title text NOT NULL,
    speech_date date NOT NULL,
    speech_text text NOT NULL,
    search_speech_text tsvector,
    CONSTRAINT speech_key PRIMARY KEY (
        president, speech_date)
);

COPY president_speeches (president, title,
         speech_date, speech_text)
FROM '/YourDirectory/president_speeches.csv'
WITH (FORMAT CSV, DELIMITER '|', HEADER OFF, QUOTE '@');
```

After executing the query, run `SELECT * FROM president_speeches;` to see the data. In pgAdmin, double-click any cell to see extra words not visible in the results grid, you should see a sizable amount of text in each row of the `speech_text` column.

<img src = "Creating & Filling the president_speeches Table.png" width = "600" style = "margin:auto"/>

Next, we'll use an `UPDATE` query to copy the contents of `speech_text` to the `tsvector` column `search_speech_text` & transform it to that data type at the same time:

```
UPDATE president_speeches
SET search_speech_text = to_tsvector(
        'english', speech_text);
```

The `SET` clause fills `search_speech_text` with the output of `to_tsvector()`. The first argument in the function specifies the language for parsing lexemes. We're using `english` here, but you can substitute `spanish`, `german`, `french`, & other languages. The second argument is the name of the input column. Run the code to fill the column.

Finally, we want to index the `search_speech_text` column to speed up searches. We know that by default, PostgreSQL's index type is B-tree. However, for full-text search, the PostgreSQL documentation recommends using the *generalised inverted index* (GIN). A GIN index, according to the documentation, contains "an index entry for each word (lexeme), with a compressed list of matching locations.

You can add a GIN index using `CREATE INDEX`:

```
CREATE INDEX search_idx ON president_speeches
USING gin(search_speech_text);
```

Now, you can use search functions.

## Searching Speech Text

Nearly 80 years' worth of presidential speeches is fertile ground for exploring history. For example, query below lists the speeches in which the president discussed Vietnam.

```
SELECT president, speech_date
FROM president_speeches
WHERE search_speech_text @@ to_tsquery(
          'english', 'Vietnam')
ORDER BY speech_date;
```

In the `WHERE` clause, the query uses the double at sign (`@@`) match operator between the `search_speech_text` column (of data type `tsvector`) & the query term *Vietnam*, which `to_tdquery` transforms into `tsquery` data. The results should list 19 speeches, showing that the first mention of Vietnam came up in a 1961 message to Congress by John F. Kennedy & because a recurring topic starting in 1966 as America's involvment in the Vietnam War escalated.

<img src = "Finding Speeches Containing the Word Vietnam.png" width = "600" style = "margin:auto"/>

### Showing Search Result Locations

To show where our search terms appear in text, we can use the `ts_headline()` function. It displays one or more highlighted search terms surrounded by adjacent words with options to format the displays one or more highlighted search terms surrounded by adjacent words with options to format the display, the number of words to show around the matched search term, & how many matched results to show from each row of text. The query below highlights how to display a search for a specific instance of the word *tax* using `ts_headline()`.

```
SELECT president,
       speech_date,
       ts_headline(speech_text, to_tsquery('english',
           'tax'), 'StartSel = <, StopSel = >,
           MinWords = 5, MaxWords = 7,
           MaxFragments = 1')
FROM president_speeches
WHERE search_speech_text @@ to_tsquery('english', 'tax')
ORDER BY speech_date;
```

To declare `ts_headline()`, we pass the original `speech_text` column rather than the `tsvector` column we used in the search function as the first argument. Then, as the second argument, we pass a `to_tsquery()` function that specifies the word to highlight.We follow this with a third argument that lists optional formatting parameters separated by commas. Here, we specify characters that will identify the start & end of the matched search term or terms (`StartSel` & `StopSel`). We also set the minimum & maximum number of total words to display, including the matched terms (`MinWords` & `MaxWords`), plus the maximum number of fragments (or instances of a match) to show using `MaxFragments` These settings are optional, & you can adjust them according to your needs.

The results of this query should show at most seven words per speech, highlighting words in which *tax* is the root:

<img src = "Displaying Search Results with ts_headline().png" width = "600" style = "margin:auto"/>

Now, we can quickly see the context of the term we searched. You might also use this function to provide flexible display options for a search feature on a web application. Notice that we didn't just find exact matches. The search engine identified `tax` along with `taxes`, `Tax`, `Taxes` & `TAX` -- words with *tax* as the root & regardless of case.

### Using Multiple Search Terms

As another example, we could look for speeches in which a president mentioned the word *transportation* but didn't discuss *roads*. We might want to do this to find speeches that focused on broader policy rather than a specific roads program. To do this, we use the syntax in the below query:

```
SELECT president,
       speech_date,
       ts_headline(speech_text, to_tsquery('english',
           'transportation & !roads'), 'StartSel = <,
           StopSel = >, MinWords = 5, MaxWords = 7,
           MaxFragments = 1')
FROM presidents_speeches
WHERE search_speech_text @@ to_tsquery('english',
          'transportation & !roads')
ORDER BY speech_date;
```

Again, we use `ts_headline()` to highlight the terms our search finds. In the `to_tsquery()` function in the `WHERE` clause, we pass `transportation` & `roads` combining them with the ampersand (`&`) operator. We use the exclamation point (`!`) in front of `roads` to indicate that we want speeches that do not contain this word. This query should find 15 speeches that fit the criteria. 

<img src = "Finding Speeches with the Word Transportation but not Roads.png" width = "600" style = "margin:auto"/>

Notice that the highlighted words in the `ts_headline` column include `transportation` & `transport`. Again, `to_tsquery()` converted `transportation` to the lexeme `transport` for the search term. This database behavior is extremely useful in helping to find relevant related words.

### Searching for Adjacent Words

Finally, we'll use the distance (`<->`) operator, which consists of a hyphen between the less-than & greater_than signs, to find adjacent words. Alternatively, you can place a number between the signs to find terms that many words apart. For example, the query below searches for any speeches that include the word *military* immediately followed by *defense*.

```
SELECT president,
       speech_date,
       ts_headline(speech_text, to_tsquery('english',
           'military <-> defense'), 'StartSel = <,
           StopSel = >, MinWords = 5, MaxWords = 7,
           MaxFragments = 1')
FROM president_speeches
WHERE search_speech_text @@ to_tsquery('english',
          'military <-> defense')
ORDER BY speech_date;
```

This query should find five speeches, & because `to_tsquery()` converts the search terms to lexemes, the words identifies in the speeches should include plurals, such as *military defenses*. The result shows the speeches that have the adjacent terms:

<img src = "Finding Speeches Where Defense Follows Military.png" width = "600" style = "margin:auto"/>

If you changed the query terms to `military <2> defense`, the database would return matches where the terms are exactly two words apart, as in the phrase "our military & defense commitments".

## Ranking Query Matches by Relevance

Youc an also rank search results by relevance using two of PostgreSQL's full-text search functions. These functions are helpful when you're trying to understand which piece of text, or speech in this case, is most relevant to your particular search terms.

One function, `ts_rank()`, generates a rank value (returned as a variable-precision `real` data type) based on how often the lexemes you're searching for appear in the text. The other function, `ts_rank_cd()`, considers how close the lexemes searched are to each other. Both functions can take optional arguments to consider document length & other factors. The rank value they generate is an arbitrary decimal that's useful for sorting but doesn't have any inherent meaning. For example, a value of `0.375` generated during one query isn't directly comparable to the same value generated during a different query.

As an example, the query below uses `ts_rank()` to rank speeches containing all the words *war*, *security*, *threat*, & *enemy*.

```
SELECT president,
       speech_date,
       ts_rank(search_speech_text, to_tsquery('english',
           'war & security & threat & enemy')) AS score
FROM president_speeches
WHERE search_speech_text @@ to_tsquery('english',
          'war & security & threat & enemy')
ORDER BY score DESC
LIMIT 5;
```

In this query, the `ts_rank()` function takes two arguments: the `search_speech_text` column & the output of a `to_tsquery()` function containing the search terms. The output of the function receives the alias `score`. In the `WHERE` clause we filter the results to only those speeches that contain the search terms specified. Then we order the results in `score` in descending order & return just five of the highest-ranking speeches. The results are as follows:

<img src = "Scoring Relevance with ts_rank().png" width = "600" style = "margin:auto"/>

Bill Clinton's 1997 State of the Union message contains the words *war*, *security*, *threat*, & *enemy* more often than the other speeches, as he discussed the Cold War & other topics. However, it also happens to be one of the longer speeches in the table (which you can determine by using `char_length()`). The lengths of speeches influences these ranking because `ts_rank()` factors in the number of matching terms in a given text. Two speeches by George W. Bush, delievered in the years before & after the start of the Iraq War, rank next.

It would be ideal to compare frequencies between speeches of identicaly lengths to get a more accurate ranking, but this isn't always possible. However, we can factor in the length of each speech by adding a normalisation code as a third parameter of the `ts_rank()` function, as shown in the query below:

```
SELECT president,
       speech_date,
       ts_rank(search_speech_text,
           to_tsquery('english',
           'war & security & threat & enemy'),
           2)::numeric AS score
FROM president_speeches
WHERE search_speech_text @@ to_tsquery('english',
          'war & security & threat & enemy')
ORDER BY score DESC
LIMIT 5;
```

Adding the optional `2` instructs the function to divide the `score` by the length of the data in the `search_speech_text` column. This quotient then represents a score normalised by the document length, giving an apples-to-apples comparison among the speeches. The rankings should change.

<img src = "Normalising ts_rank() by Speech Length.png" width = "600" style = "margin:auto"/>

In constrast to the ranking results, George W. Bush's 2004 speech now tops the ranking, & Truman's 1946 message falls out of the top five. This might be a more meaningful ranking than the first sample output, because we normalised it by length. But three of the five top-ranked speeches are the same between the two sets, & you can be reasonably certain that each of these three is worthy of closer examination to understand more about presidential speeches that include wartime terminology.

---

# Wrapping Up

Text offers abundant opportunities for data anlysis. We learned techniques for turning ordinary text into data you can extract, quantify, search, & rank, using regular expressions to dig out facts buried inside chunks of text.